# Simulated annealing approach

In [15]:
# essentials
import os
import pathlib
from copy import copy
import json
from pprint import pprint
import heapq
import time
import datetime

import pandas as pd
import numpy as np
from tqdm import tqdm

# visualisation
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import networkx as nx

In [16]:
IN_KAGGLE = False

kaggle_folder = "/kaggle/input/"
local_folder = "./data/"
puzzle_info_df = pd.read_csv(kaggle_folder if IN_KAGGLE else local_folder + "santa-2023/puzzle_info.csv")
puzzles_df = pd.read_csv(kaggle_folder if IN_KAGGLE else local_folder + "santa-2023/puzzles.csv")
sample_submission_df = pd.read_csv(kaggle_folder if IN_KAGGLE else local_folder + "santa-2023/sample_submission.csv")

# initial parsing
puzzles_df['parsed_initial_state'] = puzzles_df['initial_state'].apply(lambda x: x.split(';'))
puzzles_df['parsed_solution_state'] = puzzles_df['solution_state'].apply(lambda x: x.split(';'))
puzzle_info_df['allowed_moves'] = puzzle_info_df['allowed_moves'].apply(lambda x: json.loads(x.replace("'", '"')))

In [17]:
class PuzzleStateMachine:
    def __init__(self, puzzle_type, initial_state, solution_state, allowed_moves):
        print(f'Initializing puzzle state machine for {puzzle_type}.')
        self.puzzle_type = puzzle_type
        self.initial_state = initial_state
        self.solution_state = solution_state
        self.allowed_moves = allowed_moves
        self.state = initial_state
        self.history = [initial_state]

    def __repr__(self):
        return f'PuzzleStateMachine({self.puzzle_type}, {len(self.history)} moves)'
    
    def check_is_solved(self):
        return self.state == self.solution_state

    def check_allowed_move(self, move):
        if move not in self.allowed_moves.values():
            raise ValueError(f'Invalid move number {len(self.history)}: {move}')
    
    def inverse_move(self, move):
        return {v: k for k, v in enumerate(move)}
    
    def apply_move(self, move, inverse=False):
        """
        Apply a move or its inverse to the puzzle state.

        :param move: List representing the move as a permutation.
        :param inverse: Boolean indicating whether to apply the inverse of the move.
        """
        self.check_allowed_move(move)
        if inverse:
            inverse_move = self.inverse_move(move)
            self.state = [self.state[inverse_move[i]] for i in range(len(self.state))]
        else:
            self.state = [self.state[i] for i in move]
        self.history.append(self.state)
        
    def revert_move(self):
        if len(self.history) > 1:
            self.history.pop()
            self.state = self.history[-1]
        else:
            raise ValueError('Cannot revert initial state')

In [18]:
def apply_move(state, move, inverse=False):
    """
    Apply a move or its inverse to the puzzle state.

    :param state: List of colors representing the current state of the puzzle.
    :param move: List representing the move as a permutation.
    :param inverse: Boolean indicating whether to apply the inverse of the move.
    :return: New state of the puzzle after applying the move.
    """
    if inverse:
        # Creating a dictionary to map the original positions to the new positions
        inverse_move = {v: k for k, v in enumerate(move)}
        try:
            return [state[inverse_move[i]] for i in range(len(state))]
        except Exception as e:
            print(f"Len state: {len(state)}, len inverse_move: {len(inverse_move)}")
            raise e
    else:
        return [state[i] for i in move]
    
def improved_heuristic_with_wildcards(state, goal_state, num_wildcards):
    """
    Improved heuristic function considering wildcards.
    """
    mismatches = sum(s != g for s, g in zip(state, goal_state))
    return max(0, mismatches - num_wildcards)

def improved_a_star_search_with_wildcards(puzzle, num_wildcards, max_depth=30, timeout=100):
    """
    Improved A* search algorithm with wildcards, depth limit, and timeout.

    :param initial_state: List representing the initial state of the puzzle.
    :param goal_state: List representing the goal state of the puzzle.
    :param allowed_moves: Dictionary of allowed moves and their corresponding permutations.
    :param num_wildcards: Number of wildcards allowed for the puzzle.
    :param max_depth: Maximum depth to search to limit the search space.
    :param timeout: Time limit in seconds for the search.
    :return: Shortest sequence of moves to solve the puzzle, or None if no solution is found.
    """

    # unpack puzzle object
    initial_state = puzzle.initial_state
    goal_state = puzzle.solution_state
    allowed_moves = puzzle.allowed_moves
    
    start_time = time.time()
    open_set = []
    heapq.heappush(open_set, (0, initial_state, [], num_wildcards))  # (priority, state, path, remaining wildcards)
    closed_set = set()

    while open_set:
        if time.time() - start_time > timeout:
            return None  # Timeout

        _, current_state, path, remaining_wildcards = heapq.heappop(open_set)

        if len(path) > max_depth:  # Depth limit
            continue

        if current_state == goal_state or improved_heuristic_with_wildcards(current_state, goal_state, remaining_wildcards) == 0:
            return path

        closed_set.add((tuple(current_state), remaining_wildcards))

        for move_name, move in allowed_moves.items():
            for inverse in [False, True]:
                new_state = apply_move(current_state, move, inverse)
                if (tuple(new_state), remaining_wildcards) not in closed_set:
                    priority = len(path) + 1 + improved_heuristic_with_wildcards(new_state, goal_state, remaining_wildcards)
                    heapq.heappush(open_set, (priority, new_state, path + [(move_name, inverse)], remaining_wildcards))

    return None  # No solution found

In [21]:
def format_solution_for_submission(puzzle_id, solution_moves):
    """
    Format the solution to a puzzle for submission.

    :param puzzle_id: The unique identifier of the puzzle.
    :param solution_moves: List of tuples representing the solution moves.
    :return: Formatted string suitable for submission.
    """
    formatted_moves = []
    for move, inverse in solution_moves:
        move_str = '-' + move if inverse else move
        formatted_moves.append(move_str)

    # Joining the moves into a single string separated by periods
    return {'id': puzzle_id, 'moves': '.'.join(formatted_moves)}

def solve_row(row):
    if row['solve_from_scratch']:
        allowed_moves = puzzle_info_df[puzzle_info_df['puzzle_type'] == row['puzzle_type']]['allowed_moves'].iloc[0]
        return improved_a_star_search_with_wildcards(
                PuzzleStateMachine(row['puzzle_type'], row['parsed_initial_state'], row['parsed_solution_state'], allowed_moves),
                row['num_wildcards']
            )
    else:
        return None

def solve_puzzles(puzzles_df, puzzle_info_df, sample_submission_df, num_puzzles=None, limit_index=30):
    """
    Solve a set of puzzles using the A* search algorithm.

    :param puzzles_df: DataFrame containing puzzles.
    :param puzzle_info_df: DataFrame containing allowed moves for each puzzle type.
    :param sample_submission_df: DataFrame containing sample submission format.
    :param num_puzzles: Number of puzzles to solve (if None, solve all).
    :return: DataFrame with the solutions formatted for submission.
    """
    solutions = []

    # limit the number of puzzles if specified
    puzzles_to_solve = puzzles_df if num_puzzles is None else puzzles_df.head(num_puzzles)

    #puzzles_to_solve['solve_from_scratch'] = puzzles_to_solve.index <= limit_index
    #puzzles_to_solve['solution_moves'] = puzzles_to_solve.apply(solve_row, axis=1)
    #puzzles_to_solve['formatted_solution'] = puzzles_to_solve['solution_moves'].apply(
    #    lambda x: format_solution_for_submission(x[0], x[1]) if x is not None else None
    #)
    #for index, row in tqdm(puzzles_to_solve.iterrows(), total=puzzles_to_solve.shape[0], desc="Solving Puzzles"):
    #    solutions.append(row['formatted_solution'])


    for index, row in tqdm(puzzles_to_solve.iterrows(), total=puzzles_to_solve.shape[0], desc="Solving Puzzles"):
        puzzle_id = row['id']
        initial_state = row['parsed_initial_state']
        solution_state = row['parsed_solution_state']
        puzzle_type = row['puzzle_type']
        num_wildcards = row['num_wildcards']
        allowed_moves = puzzle_info_df[puzzle_info_df['puzzle_type'] == puzzle_type]['allowed_moves'].iloc[0]

        solution_moves = None
        if index < limit_index:
            solution_moves = improved_a_star_search_with_wildcards(
                PuzzleStateMachine(puzzle_type, initial_state, solution_state, allowed_moves),
                num_wildcards
            )
        # if no solution found, 
        if solution_moves is None:
            solution_moves = sample_submission_df[
                sample_submission_df['id'] == puzzle_id
            ]['moves'].iloc[0].split('.')
            solution_moves = [(move.replace('-', ''), move.startswith('-')) for move in solution_moves]

        formatted_solution = format_solution_for_submission(puzzle_id, solution_moves)
        solutions.append(formatted_solution)

    return pd.DataFrame(solutions)

# Solving the puzzles
solved_puzzles_df = solve_puzzles(puzzles_df, puzzle_info_df, sample_submission_df, limit_index=3)
solved_puzzles_df

Solving Puzzles:   1%|          | 2/398 [00:00<00:26, 15.08it/s]

Initializing puzzle state machine for cube_2/2/2.
Initializing puzzle state machine for cube_2/2/2.
Initializing puzzle state machine for cube_2/2/2.


Solving Puzzles: 100%|██████████| 398/398 [00:05<00:00, 68.69it/s] 


,id,moves
0,0,r1.-f1
1,1,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0
2,2,-d1.-r0.f0.-r1.f1.d1.-r1.-f0.d1.f0.d1.d1
3,3,-f0.-r0.-f0.-d0.-f0.f1.r0.-d1.-r0.-r1.-r0.-f1....
4,4,d1.-f1.d1.r1.-f0.d1.-d0.-r1.d1.d1.-f1.d1.-d0.-...
...,...,...
393,393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....
396,396,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...
